# Une fonction pour simplifier l'accès aux données

In [1]:
def extractField(s: String, fieldNumber: Int): String = {
    val fields = s.split('\t')
    if (fieldNumber >= fields.length) "" else fields(fieldNumber)
}

In [2]:
println(extractField("2	CASSIOPEE	2009	33	3.0000", 0))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 1))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 2))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 3))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 4))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 5))

2
CASSIOPEE
2009
33
3.0000


# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [3]:
val lignes = sc.textFile("prenoms_sample.txt")
lignes.take(10).foreach(println)

2	LISON	2004	69	10.0000
2	LISON	2011	59	59.0000
2	LISON	2012	86	4.0000
2	LISON	2014	90	3.0000
2	LIVIA	2002	94	4.0000
2	LIYA	2011	62	3.0000
2	LIYAH	2010	93	6.0000
2	LIZ�A	2014	76	3.0000
2	LOANE	2008	37	12.0000
2	LOLA	2006	11	23.0000


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD `prenoms` à partir de `lignes`

In [4]:
val prenoms = lignes.map(l => (
    extractField(l, 0).charAt(0),
    extractField(l, 1),
    extractField(l, 2).toInt,
    extractField(l, 3).toInt,
    extractField(l, 4).toDouble.toInt
))
prenoms.take(10).foreach(println)

(2,LISON,2004,69,10)
(2,LISON,2011,59,59)
(2,LISON,2012,86,4)
(2,LISON,2014,90,3)
(2,LIVIA,2002,94,4)
(2,LIYA,2011,62,3)
(2,LIYAH,2010,93,6)
(2,LIZ�A,2014,76,3)
(2,LOANE,2008,37,12)
(2,LOLA,2006,11,23)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD), [PairRDDFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)).

1. Rappeler ce que sont les *transformations* et les *actions*
1. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [5]:
val effectifParPrenoms = prenoms.map(p => (p._2, p._5)).reduceByKey((n1, n2) => n1 + n2)
effectifParPrenoms.take(10).foreach(println)

(BRADLEY,17)
(LISANDRO,3)
(NADIR,9)
(WISSEM,4)
(ASSIA,3)
(JENNYFER,13)
(ERNA,7)
(NICOLLE,4)
(AM�LIE,105)
(DANIELE,42)


1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [6]:
val effectifFéminin = prenoms.filter(p => p._1 == '2').map(p => p._5).sum
println(effectifFéminin)

33273.0


1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [7]:
val prenomsAvecEffectif = prenoms.map(p => (p._2, (p._5, p._5)))
def minMax(p1: Tuple2[Int, Int], p2: Tuple2[Int, Int]): Tuple2[Int, Int] = (math.min(p1._1, p2._1), math.max(p1._2, p2._2))
val minMaxParPrenoms = prenomsAvecEffectif.aggregateByKey((Integer.MAX_VALUE, Integer.MIN_VALUE))(minMax, minMax)
minMaxParPrenoms.take(10).foreach(println)

(BRADLEY,(17,17))
(LISANDRO,(3,3))
(NADIR,(3,6))
(WISSEM,(4,4))
(ASSIA,(3,3))
(JENNYFER,(6,7))
(ERNA,(7,7))
(NICOLLE,(4,4))
(AM�LIE,(5,62))
(DANIELE,(8,34))


1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [9]:
val lignesDept = sc.textFile("dpts.txt").filter(l => l.startsWith("REGION") == false)
val depts = lignesDept.map(l => (
    extractField(l, 0).toInt,
    extractField(l, 1),
    extractField(l, 2),
    extractField(l, 3).toInt,
    extractField(l, 4),
    extractField(l, 5)
))
depts.takeSample(false, 10)

Array((94,2A,2A004,3,CORSE-DU-SUD,Corse-du-Sud), (11,91,91228,5,ESSONNE,Essonne), (76,09,09122,5,ARIEGE,Ari�ge), (44,88,88160,4,VOSGES,Vosges), (84,74,74010,3,HAUTE-SAVOIE,Haute-Savoie), (24,41,41018,2,LOIR-ET-CHER,Loir-et-Cher), (76,30,30189,2,GARD,Gard), (84,15,15014,2,CANTAL,Cantal), (52,44,44109,3,LOIRE-ATLANTIQUE,Loire-Atlantique), (52,85,85191,3,VENDEE,Vend�e))

In [10]:
val prenomsParDepts = prenoms.filter(p => p._3 >= 2000).map(p => (p._4, (p._2, p._5)))
val maxParDepts = prenomsParDepts.reduceByKey((p1, p2) => (if (p1._2 < p2._2) p2._1 else p1._1, math.max(p1._2, p2._2)))
val deptsAvecNoms = depts.filter(d => d._2 != "2A" && d._2 != "2B").map(d => (d._2.toInt, d._5))
val resultats = deptsAvecNoms.join(maxParDepts).map(j => (j._2._1, (j._2._2._1, j._2._2._2))).sortByKey()
resultats.collect.foreach(println)

(AIN,(MAXIME,18))
(AISNE,(JUSTINE,23))
(ALLIER,(ANTOINE,15))
(ALPES-DE-HAUTE-PROVENCE,(LENNY,5))
(ALPES-MARITIMES,(MAXENCE,46))
(ARDECHE,(ESTEBAN,15))
(ARDENNES,(SABRINA,3))
(ARIEGE,(ARTHUR,7))
(AUBE,(BAPTISTE,23))
(AUDE,(CL�MENT,25))
(AVEYRON,(M�LANIE,11))
(BAS-RHIN,(MANON,64))
(BOUCHES-DU-RHONE,(LENNY,69))
(CALVADOS,(ALYSSIA,7))
(CANTAL,(ROMANE,14))
(CHARENTE,(TOM,21))
(CHARENTE-MARITIME,(MAXIME,41))
(CHER,(MAEL,11))
(CORREZE,(LUCAS,32))
(COTE-D'OR,(_PRENOMS_RARES,164))
(COTES-D'ARMOR,(TITOUAN,27))
(CREUSE,(MAXIME,3))
(DEUX-SEVRES,(ENOLA,7))
(DORDOGNE,(ENZO,54))
(DOUBS,(ANNA,23))
(DROME,(LUCIE,11))
(ESSONNE,(SARAH,84))
(EURE,(VICTOR,14))
(EURE-ET-LOIR,(MARIE,12))
(FINISTERE,(_PRENOMS_RARES,207))
(GARD,(MATH�O,37))
(GERS,(VICTOIRE,3))
(GIRONDE,(L�A,60))
(HAUT-RHIN,(SIMON,20))
(HAUTE-GARONNE,(BASTIEN,48))
(HAUTE-LOIRE,(PAUL,4))
(HAUTE-MARNE,(EMMA,23))
(HAUTE-SAONE,(MAELYS,5))
(HAUTE-SAVOIE,(MARTIN,31))
(HAUTE-VIENNE,(ROXANE,6))
(HAUTES-ALPES,(ARTHUR,5))
(HAUTES-PYRENEES,(CL�MENT,21))
(